In [ ]:
# Importation des modules

import pandas as pd
from gensim.models import Word2Vec
import random
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

In [ ]:
def moyenne(L):
    s = 0
    for i in L:
        if str(i) == 'inf':
            pass
        else:
            s += i
        
    return s/len(L)

In [ ]:
w2v_model_novice = Word2Vec.load("results\word2vec_novice_300.model")
w2v_model_exp = Word2Vec.load("results\word2vec_exp_300.model")
w2v_model_all = Word2Vec.load("results\word2vec_all_300.model")

In [ ]:
words_novice = list(w2v_model_novice.wv.vocab.keys())
words_exp = list(w2v_model_exp.wv.vocab.keys())
words_all = list(w2v_model_all.wv.vocab.keys())

In [ ]:
#v = w2v_model_novice.wv.get_vector('ministre') + w2v_model_novice.wv.get_vector('santé')
#w2v_model_novice.similar_by_vector(v, topn=10, restrict_vocab=None)

In [ ]:
w2v_model_all.wv.most_similar(positive=["macron_Exp"])

### Overlap et spread

#### Overlap

In [ ]:
def overlap(model1, model2, n, word):

    neighborhood_1 = set([i[0] for i in model1.wv.most_similar(positive=[word], topn = n)])
    neighborhood_2 = set([i[0] for i in model2.wv.most_similar(positive=[word], topn = n)])

    return len(neighborhood_1.intersection(neighborhood_2))

In [ ]:
words_common = list(set(words_exp).intersection(set(words_novice)))
random.shuffle(words_common)

In [ ]:
# Overlap avec une profondeur de 10 mots

L_10 = [overlap(w2v_model_novice, w2v_model_exp, 10, word) for word in words_common[:2000]]
plt.plot([np.mean(L_10[:i]) for i in range(2000)])

In [ ]:
# Overlap avec une profondeur de 100 mots


L_100 = [overlap(w2v_model_novice, w2v_model_exp, 100, word) for word in words_common[:10000]]
plt.plot([np.mean(L_100[:i]) for i in range(10000)])

In [ ]:
# Overlap avec une profondeur de 1000 mots

L_1000 = [overlap(w2v_model_novice, w2v_model_exp, 1000, word) for word in words_common[:10000]]
plt.plot([np.mean(L_1000[:i]) for i in range(10000)])

#### Spread

In [ ]:
def rank(model, i, j):
    
    k = -1
    K = 10
    
    while k == -1 and 10 * K < len(words_exp):
        neighborhood = [i[0] for i in model.wv.most_similar(positive=[i], topn = K)]
        try:
            k = neighborhood.index(j)
        except:
            k = -1
        K *= 10
            
    return k + 1

def spread(model1, model2, n, word):

    neighborhood_1 = [i[0] for i in model1.wv.most_similar(positive=[word], topn = n)]
    ranks = [rank(model2, word, j) for j in neighborhood_1]
    return max(ranks)

In [ ]:
#print(w2v_model_exp.wv.most_similar(positive=['droite'], topn = 10))
#rank(w2v_model_exp, 'droite', 'sarkozysme')

In [ ]:
spread(w2v_model_novice, w2v_model_exp, 10, 'droite')

In [ ]:
plt.plot([spread(w2v_model_novice, w2v_model_exp, i, 'demain') for i in range(1,100)])

In [ ]:
plt.plot([spread(w2v_model_novice, w2v_model_exp, 6, word) for word in words_common[:100]])

## WMD 

In [ ]:
# Normalisation des vecteurs

w2v_model_exp.init_sims(replace = True)
w2v_model_novice.init_sims(replace = True)
w2v_model_all.init_sims(replace = True)

In [ ]:
df0 = pd.read_csv('data/data_cleaned_NLP.csv', sep = ',', encoding = 'latin-1')

In [ ]:
df1 = df0[['Réplique', 'Groupe', 'tokenized_replique', 'Orateur']].copy()

df_novice = df1[df1['Groupe'] == 'Novice'].copy()
df_exp = df1[df1['Groupe'] == 'Exp'].copy()

In [ ]:
df_novice = df_novice.dropna()

df_exp = df_exp.dropna()

In [ ]:
phrase0 = df_novice.iloc[0]['tokenized_replique'].split(' ')
phrase1 = df_novice.iloc[1]['tokenized_replique'].split(' ')
phrase2 = df_novice.iloc[2]['tokenized_replique'].split(' ')

### Calcul des distances intra

#### Novices

##### Distance selon le modèle Novice

In [ ]:
df_novice_shuffled = df_novice.sample(frac=1)
phrases_novice = [df_novice_shuffled.iloc[i]['tokenized_replique'].split(' ') for i in range(2 * 10000)]

In [ ]:
WMD_novices = [w2v_model_novice.wv.wmdistance(phrases_novice[i], phrases_novice[i + 1]) for i in range(0, 20000, 2)]

In [ ]:
means_novice = [moyenne(WMD_novices[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_novice)

In [ ]:
distance_intra_novice = np.mean(means_novice[9000:10000])
print(distance_intra_novice)

##### Distance selon le modèle Exp


In [ ]:
WMD_novices_expmodel = [w2v_model_exp.wv.wmdistance(phrases_novice[i], phrases_novice[i + 1]) for i in range(0, 20000, 2)]

In [ ]:
means_novice_expmodel = [moyenne(WMD_novices_expmodel[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_novice_expmodel)

In [ ]:
distance_intra_novice_expmodel = np.mean(means_novice_expmodel[9000:10000])
print(distance_intra_novice_expmodel)

##### Distance selon le modèle All

In [ ]:
def add_groupe(L, etiquette):
    return [i + etiquette for i in L]

In [ ]:
WMD_novices_allmodel = [
    w2v_model_all.wv.wmdistance(
            add_groupe(phrases_novice[i], '_Novice'), 
            add_groupe(phrases_novice[i + 1], '_Novice')) 
    for i in range(0, 20000, 2)
                        ]

In [ ]:
means_novice_allmodel = [moyenne(WMD_novices_allmodel[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_novice_allmodel)

In [ ]:
distance_intra_novice_allmodel = np.mean(means_novice_allmodel[9000:10000])
print(distance_intra_novice_allmodel)

#### Non-novices

##### Distance selon le modèle Exp

In [ ]:
df_exp_shuffled = df_exp.sample(frac = 1)
phrases_exp = [df_exp_shuffled.iloc[i]['tokenized_replique'].split(' ') for i in range(2 * 10000)]

In [ ]:
WMD_exp = [w2v_model_exp.wv.wmdistance(phrases_exp[i], phrases_exp[i + 1]) for i in range(0, 20000, 2)]

In [ ]:
means_exp = [moyenne(WMD_exp[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_exp)

In [ ]:
distance_intra_exp = np.mean(means_exp[9000:10000])
print(distance_intra_exp)

##### Distance selon le modèle Novice

In [ ]:
WMD_exp_novicemodel = [w2v_model_novice.wv.wmdistance(phrases_exp[i], phrases_exp[i + 1]) for i in range(0, 20000, 2)]

In [ ]:
means_exp_novicemodel = [moyenne(WMD_exp_novicemodel[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_exp_novicemodel)

In [ ]:
distance_intra_exp_novicemodel = np.mean(means_exp_novicemodel[9000:10000])
print(distance_intra_exp_novicemodel)

##### Distance selon le modèle All

In [ ]:
WMD_exp_allmodel = [
    w2v_model_all.wv.wmdistance(
            add_groupe(phrases_exp[i], '_Exp'), 
            add_groupe(phrases_exp[i + 1], '_Exp')) 
    for i in range(0, 20000, 2)
                        ]

In [ ]:
means_exp_allmodel = [moyenne(WMD_exp_allmodel[:i]) for i in range(1, 10000)]

In [ ]:
plt.plot([i for i in range(1, 10000)], means_exp_allmodel)

In [ ]:
distance_intra_exp_allmodel = np.mean(means_exp_allmodel[9000:10000])
print(distance_intra_exp_allmodel)

### Distances inter

In [ ]:
WMD_inter_model_exp = [w2v_model_exp.wmdistance(phrases_exp[i], phrases_novice[i]) for i in range(0, 600)]
WMD_inter_model_novice = [w2v_model_novice.wmdistance(phrases_exp[i], phrases_novice[i]) for i in range(0, 600)]
WMD_inter_model_all = [w2v_model_all.wmdistance(
                                        add_groupe(phrases_exp[i], '_Exp'),
                                        add_groupe(phrases_novice[i], '_Novice'))
                                       for i in range(0, 600)]

In [ ]:
means_model_exp = [moyenne(WMD_inter_model_exp[:i]) for i in range(1, 600)]
means_model_novice = [moyenne(WMD_inter_model_novice[:i]) for i in range(1, 600)]
means_model_all = [moyenne(WMD_inter_model_all[:i]) for i in range(1, 600)]

In [ ]:
plt.plot([i for i in range(600)], WMD_inter_model_novice, alpha = .2)
plt.plot([i for i in range(1, 600)], means_model_novice)

In [ ]:
distance_inter_novice = np.mean(means_model_novice[300:600])
print(distance_inter_novice)

In [ ]:
plt.plot([i for i in range(600)], WMD_inter_model_exp, alpha = .2)
plt.plot([i for i in range(1, 600)], means_model_exp)

In [ ]:
distance_inter_exp = np.mean(means_model_exp[300:500])
print(distance_inter_exp)

In [ ]:
plt.plot([i for i in range(600)], WMD_inter_model_all, alpha = .2)
plt.plot([i for i in range(1, 600)], means_model_all)

In [ ]:
distance_inter_all = np.mean(means_model_exp[300:500])
print(distance_inter_exp)

### Résumé WMD

In [ ]:
scores = [distance_intra_novice, distance_intra_exp_novicemodel, distance_inter_novice,
          distance_intra_novice_expmodel, distance_intra_exp, distance_inter_exp,
          distance_intra_novice_allmodel, distance_intra_exp_allmodel, distance_inter_all]

In [ ]:
scores = [1.272288065714167, 1.2800969970269758, 1.271612526520287,
          1.2712417147548438, 1.2904593128668091, 1.2845069306011527,
          1.1111787841815146, 1.2111982233569398, distance_inter_all]

In [ ]:
px.histogram(x = ['Intra novice, modèle Novice', 'Intra Exp, modèle Novice', 'Inter, modèle novice',
                  'Intra Novice, modèle Exp', 'Intra Exp, modèle Exp', 'Inter, modèle Exp',
                  'Intra Novice, modèle All', 'Intra Exp, modèle All', 'Inter, modèle All'
                   ], 
             y = scores,
             range_y = [0.995 * min(scores), 1.005 * max(scores)])